In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import re
import string
from nltk.corpus import stopwords

import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm

# 1. Newsoutlets

In [2]:
outlets = pd.read_csv('Newsoutlets.csv')
outlets.head(7)

,newsoutlet_id,newsoutlet_name,newsoutlet_country
0,53587,"""CNN Breaking News""",USA
1,63496,"""The New York Times""",USA
2,67332,"""CNN""",USA
3,68616,"""CBS News""",USA
4,69081,"""The Wall Street Journal""",USA
5,69340,"""NPR""",USA
6,69373,"""The Washington Post""",SA


In [3]:
outlets.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27 entries, 0 to 26
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   newsoutlet_id        27 non-null     int64 
 1    newsoutlet_name     27 non-null     object
 2    newsoutlet_country  27 non-null     object
dtypes: int64(1), object(2)
memory usage: 776.0+ bytes


In [4]:
# Alteração do nome das colunas ' newsoutlet_name', ' newsoutlet_country' retirando o espaço inicial
outlets.rename(columns={' newsoutlet_name': 'newsoutlet_name', 
                        ' newsoutlet_country': 'newsoutlet_country'}, inplace=True)

Verificamos que os 'newsoutlet_id's não correspondem ao indicado no README do dataset e consequentemente não é possível fazer merge a partir desse atributo. Vamos corrigir manualmente visto serem poucos registos.

In [5]:
outlets.replace([53587,63496,67332,68616,69081,69340,69373,71071,71311,71562,
                  72299,72466,73039,73047,72284,72760,72774,72933,72995,73033,
                  73052,73058,73059,73063,73064,73065,73066],
                 [428333,807095,759251,15012486,3108351,5392522,2467791,7309052,
                  28785486,15754281,14511951,14173315,95431448,1020058453,21866939,
                  271413771,16343974,111556423,138749160,87818409,612473,6107422,
                  16887175,17895820,34655603,7587032,16973333]
                ,inplace=True)

O outlet **_"The Washington Post"_** no índice 6 está com o valor do atributo *newsoutlet_country* distorcido, portanto vamos corrigir.

In [6]:
outlets.iloc[6,2]='USA'
outlets

,newsoutlet_id,newsoutlet_name,newsoutlet_country
0,428333,"""CNN Breaking News""",USA
1,807095,"""The New York Times""",USA
2,759251,"""CNN""",USA
3,15012486,"""CBS News""",USA
4,3108351,"""The Wall Street Journal""",USA
5,5392522,"""NPR""",USA
6,2467791,"""The Washington Post""",USA
7,7309052,"""Yahoo News""",USA
8,28785486,"""ABC News""",USA
9,15754281,"""USA TODAY""",USA


# 3. NewsItemsReplies

In [3]:
ireplies = pd.read_csv('NewsItemsReplies.csv', low_memory=False, na_values = ['n\a', 'na', 'None', 'NaN', '--', '.'])
ireplies.head()

,comment_id,news_tweet_text,comment_tweet_date,in_reply_to_news_id,comment_tweet_text,perspective_attack_on_author,perspective_attack_on_commenter,perspective_identity_attack,perspective_inflammatory,perspective_insult,perspective_profanity,perspective_severe_toxicity,perspective_threat,perspective_toxicity
0,1084048846781583360,People over 65 and ultra conservatives shared ...,2019-01-12 11:23:05,1084048113260773377,@ABC Probably because Conservatives are the sa...,0.004244,0.653545,0.293428,0.396587,0.445224,0.227399,0.109481,0.152440,0.386526
1,1081750348010872832,OPINION: Congresswoman Rashida Tlaib should ap...,2019-01-05 23:24:52,1081693041222709249,@ABC @matthewjdowd Have you forgotten who she ...,0.126954,0.911894,0.417152,0.650789,0.419343,0.318712,0.314918,0.459924,0.499502
2,1083125706689822720,Rep. Kevin McCarthy disputes Sen. Schumer's ac...,2019-01-09 21:39:48,1083116150660055040,@ABC Democratic Neanderthals! Who doesn’t like...,0.054733,0.385629,0.346428,0.568032,0.491160,0.367424,0.237174,0.234344,0.440851
3,1081555613098889216,OPINION: Congresswoman Rashida Tlaib should ap...,2019-01-05 11:06:13,1081507153721114629,@ABC @matthewjdowd When Trump apologizes is wh...,0.018760,0.299981,0.134293,0.226994,0.159741,0.150060,0.087283,0.157148,0.224154
4,1081279148612571137,JUST IN: Pres. Trump says he's considered decl...,2019-01-04 19:55:11,1081277886105976832,@ABC That continent is stuck with the baggage ...,0.025047,0.592016,0.330678,0.568032,0.592763,0.484796,0.341170,0.353246,0.617881


In [4]:
ireplies.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2228792 entries, 0 to 2228791
Data columns (total 14 columns):
 #   Column                           Dtype  
---  ------                           -----  
 0   comment_id                       object 
 1   news_tweet_text                  object 
 2   comment_tweet_date               object 
 3   in_reply_to_news_id              object 
 4   comment_tweet_text               object 
 5   perspective_attack_on_author     float64
 6   perspective_attack_on_commenter  float64
 7   perspective_identity_attack      float64
 8   perspective_inflammatory         float64
 9   perspective_insult               float64
 10  perspective_profanity            float64
 11  perspective_severe_toxicity      float64
 12  perspective_threat               float64
 13  perspective_toxicity             float64
dtypes: float64(9), object(5)
memory usage: 238.1+ MB


## Duplicados

In [5]:
ireplies.drop_duplicates(inplace=True)
len(ireplies)

1239325

In [9]:
## DROP_DUPLICATES CONDICIONADO ##
# Ordena IN_REPLY_TO_NEWS_ID de forma a que estejam primeiro os que estão 
# em notação cientifica e, por fim, elimina os duplicados mantendo só os 
# últimos que terão o formato normal, caso existam.

ireplies.sort_values('in_reply_to_news_id',inplace=True)
ireplies.drop_duplicates(['comment_id','news_tweet_text','comment_tweet_date',
                          'comment_tweet_text'], keep='last', inplace=True)
len(ireplies)

1114396

## Notação científica no 'in_reply_to_news_id'

In [10]:
# linhas com IN_REPLY_TO_NEWS_ID em notação cientifica
iRep2News = ireplies[ireplies[['in_reply_to_news_id']]
             .apply(lambda x: x.str.contains(',|E|e', regex=True))
             .any(axis=1)]

# nº de comentários em notação científica
len(iRep2News)

98332

### Procura das notícias correspondentes aos comentários com 'in_reply_to_news_id' em notação científica

In [11]:
sci_notation = iRep2News['in_reply_to_news_id'].value_counts()
#left most digits
lm_digits = sci_notation.index.str.replace('\.|,|[e|E]\+[0-9]*','',regex=True)
df = pd.DataFrame({'sci_notation': sci_notation.index,'lm_digits': lm_digits, '# comments': sci_notation})
df

,sci_notation,lm_digits,# comments
1.083897478e+18,1.083897478e+18,1083897478,4639
1.080788389e+18,1.080788389e+18,1080788389,2851
1.082839415e+18,1.082839415e+18,1082839415,1567
1.083918267e+18,1.083918267e+18,1083918267,1448
1.07973272e+18,1.07973272e+18,107973272,1216
...,...,...,...
1.0825248260808173e+18,1.0825248260808173e+18,10825248260808173,1
1.0802108065648065e+18,1.0802108065648065e+18,10802108065648065,1
1.0812617598628413e+18,1.0812617598628413e+18,10812617598628413,1
1.0837575371057112e+18,1.0837575371057112e+18,10837575371057112,1


In [12]:
items = pd.read_csv('NewsItems.csv', na_values = ['n\a', 'na', 'None', 'NaN', '--', '.'])

# lista de id's completos das notícias, em ITEMS, como strings para comparação
full_news_id = items['news_id'].astype(str)

# possíveis 'news_id's associados aos 'lm_digits' (left most digits) 
# que retiramos da notação científica
df2 = pd.DataFrame({'news_id' : [], 'lm_digits' : []})
    
for parcial_id in df['lm_digits']:
    regex = '^' + parcial_id + '[0-9]*'
    aux_df = full_news_id[full_news_id.str.contains(regex)].to_frame()
    aux_df['lm_digits'] = parcial_id
    df2 = pd.concat([df2, aux_df], axis=0)

df2

# De 3654 'lm_digits' apenas encontramos 1791 'news_id's compatíveis (...)

,news_id,lm_digits
14583,1082839415075061760,1082839415
14591,1083918267000016896,1083918267
14089,1079732720039849986,107973272
14297,1082480014413758466,1082480014
13573,1082577684314632192,1082577684
...,...,...
15616,1079825788822802437,10798257888228024
15634,1078731067102806016,1078731067102806
11576,1080456444133150721,10804564441331507
15953,1081261759862841344,10812617598628413


In [13]:
df2.nunique()

news_id      1781
lm_digits    1682
dtype: int64

In [14]:
# (...) desses 8 são repetidos
df2[df2.duplicated('news_id', keep=False)].sort_values(by='news_id')

,news_id,lm_digits
15875,1078319627401392135,1078319627401392
15875,1078319627401392135,107831962
14641,1080256132579508224,10802561325795082
14641,1080256132579508224,108025613
10993,1082796616191377408,1082796616
10993,1082796616191377408,10827966161913774
2740,1082887228555173888,10828872
3292,1082887228555173888,10828872
12404,1083729249985617922,10837292499856179
12404,1083729249985617922,1083729


In [15]:
# basta retirar os primeiros duplicados porque são os que têm 
# os 'lm_digits' com menor precisão
df2.drop_duplicates(subset=['news_id'], keep='last', inplace=True)
df2.nunique()

news_id      1781
lm_digits    1679
dtype: int64

In [16]:
# Informação completa dos id's restantes
m = pd.merge(df2, df, on='lm_digits', how="left")
m

,news_id,lm_digits,sci_notation,# comments
0,1082839415075061760,1082839415,1.082839415e+18,1567
1,1083918267000016896,1083918267,1.083918267e+18,1448
2,1079732720039849986,107973272,1.07973272e+18,1216
3,1082480014413758466,1082480014,1.082480014e+18,803
4,1082577684314632192,1082577684,1.082577684e+18,742
...,...,...,...,...
1776,1079825788822802437,10798257888228024,1.0798257888228024e+18,1
1777,1078731067102806016,1078731067102806,1.078731067102806e+18,1
1778,1080456444133150721,10804564441331507,1.0804564441331507e+18,1
1779,1081261759862841344,10812617598628413,1.0812617598628413e+18,1


In [17]:
# nº de comentários aproveitados
m['# comments'].sum()

57721

In [18]:
# Dicionário com as alterações para os id's no formato correcto
sciNot_dict = dict(zip(m.sci_notation, m.news_id))

# Substituição da notação cientifica pelos respectivos ids completos
ireplies.replace({'in_reply_to_news_id': sciNot_dict}, inplace=True)

In [19]:
# Comments without news #
# comentários restantes com notação científica, i.e., aos quais não é possível associar nenhum news_id
commentWTnews = ireplies[ireplies[['in_reply_to_news_id']]
                 .apply(lambda x: x.str.contains(',|E|e', regex=True))
                 .any(axis=1)]

print(f'{commentWTnews.index.size} comentários desaproveitados')

46245 comentários desaproveitados


In [20]:
# drop dos comentários que não se podem aproveitar
ireplies.drop(commentWTnews.index,inplace=True)

print(f' nº de comentários no ireplies: {ireplies.index.size}')

 nº de comentários no ireplies: 1068151


In [21]:
ireplies = ireplies.astype({'in_reply_to_news_id': int})
ireplies.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1068151 entries, 162478 to 646493
Data columns (total 14 columns):
 #   Column                           Non-Null Count    Dtype  
---  ------                           --------------    -----  
 0   comment_id                       1068151 non-null  object 
 1   news_tweet_text                  1068151 non-null  object 
 2   comment_tweet_date               1068151 non-null  object 
 3   in_reply_to_news_id              1068151 non-null  int64  
 4   comment_tweet_text               1068151 non-null  object 
 5   perspective_attack_on_author     1022560 non-null  float64
 6   perspective_attack_on_commenter  1022560 non-null  float64
 7   perspective_identity_attack      1022560 non-null  float64
 8   perspective_inflammatory         1022560 non-null  float64
 9   perspective_insult               1022560 non-null  float64
 10  perspective_profanity            1022560 non-null  float64
 11  perspective_severe_toxicity      1022560 non-n

# 2. NewsItems

In [22]:
#items = pd.read_csv('NewsItems.csv', na_values = ['n\a', 'na', 'None', 'NaN', '--', '.'])

In [23]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 43305 entries, 0 to 43304
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   news_id          43305 non-null  int64 
 1   newsoutlet_id    43305 non-null  int64 
 2   news_tweet_date  43305 non-null  object
 3   news_url         43305 non-null  object
 4   new_title        43232 non-null  object
 5   news_date        27249 non-null  object
 6   news_text        42780 non-null  object
dtypes: int64(2), object(5)
memory usage: 2.3+ MB


## Duplicados

In [24]:
# Eliminação de duplicados completos (todos os atributos)
# 552 duplicados removidos
items.drop_duplicates(inplace=True)

len(items)

42753

In [25]:
items.duplicated(['news_id', 'newsoutlet_id']).value_counts()

False    42723
True        30
dtype: int64

## Múltiplas notícias no mesmo tweet

In [26]:
len(items[items['news_id'].duplicated(keep=False) == True])

47

In [27]:
items['news_id'].value_counts().value_counts()

1    42706
3       13
2        4
Name: news_id, dtype: int64

**news_id** tem duplicados pois cada tweet pode ter várias notícias (**Decisão**: drop destes 47)

In [28]:
dup_news = items['news_id'].value_counts() > 1
dup_news_ids = dup_news.index[dup_news]

# seleciona os items com várias notícias e remove do dataset a partir do índice
dup_news_ids_idx = items[items.news_id.isin(dup_news_ids)].index
items.drop(dup_news_ids_idx, inplace=True)

len(items)

42706

## Notícias sem comentários associados

In [29]:
# valores de news_ids(aka 'in_reply_to_news_id') no ireplies
news_id_rep = ireplies['in_reply_to_news_id'].unique()

# notícias (news_ids) que estão no items mas não estão no ireplies, i.e., não têm comentários associados
news_notin_items = items[~items.news_id.isin(news_id_rep)]
news_notin_items.count()

news_id            10494
newsoutlet_id      10494
news_tweet_date    10494
news_url           10494
new_title          10493
news_date           7951
news_text          10204
dtype: int64

In [30]:
items.news_id.isin(news_id_rep).value_counts()

True     32212
False    10494
Name: news_id, dtype: int64

In [31]:
# Apenas notícias com comentários
items = items[items.news_id.isin(news_id_rep)]

len(items)

32212

## News_text

In [32]:
items['news_text'].value_counts()

About Your Privacy on this Site\nWelcome! To bring you the best content on our sites and applications, Meredith partners with third party advertisers to serve digital ads, including personalized digital ads. Those advertisers use tracking technologies to collect information about your activity on our sites and applications and across the Internet and your other apps and devices.\nYou always have the choice to experience our sites without personalized advertising based on your web browsing activity by visiting the DAA’s Consumer Choice page, the NAI's website, and/or the EU online choices page, from each of your browsers or devices. To avoid personalized advertising based on your mobile app activity, you can install the DAA’s AppChoices app here. You can find much more information about your privacy choices in our privacy policy. Even if you choose not to have your activity tracked by third parties for advertising services, you will still see non-personalized ads on our site.\nBy clicki

As notícias que aparecem mais que 27 vezes eram todas "spam", nas restantes havia notícias que eram mesmo notícias.

In [33]:
len(items)

32212

In [34]:
num_newstext=items['news_text'].value_counts()
ntmoreone = num_newstext > 27
ntmoreoneb = ntmoreone.index[ntmoreone]

#indices das notícias com texto presente em mais de 27 notíxias
ntmoreonec = items[items.news_text.isin(ntmoreoneb)].index
items.drop(ntmoreonec, inplace=True)

len(items)

30809

In [35]:
items['news_text'].value_counts()

Highlighting the efforts of some of the country's best teachers, professors, instructors, coaches and principals, who take their job to another level to help a new generation of Americans.\n1:40                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

A notícia que aparecia mais vezes (23 vezes) após eliminar as do "spam", era de facto uma notícia.

In [36]:
# Dataframe das notícias em que o 'news_text' aparece mais que uma vez.

num_newstext2=items['news_text'].value_counts()
nt_moreone = num_newstext2 > 1
nt_moreoneb = nt_moreone.index[nt_moreone]
nt_moreonec = items[items.news_text.isin(nt_moreoneb)]
nt_moreonec

,news_id,newsoutlet_id,news_tweet_date,news_url,new_title,news_date,news_text
2,1084275179331772416,28785486,2019-01-13 02:25:22,https://abcn.ws/2Fv3iri,Drone captures California man surfing with dol...,NaN,"Breathtaking drone video of a pod of friendly,..."
3,1080216134467702784,28785486,2019-01-01 21:36:10,https://abcn.ws/2CHWRj9,"In 'unprecedented' speech, Kim Jong Un says No...",NaN,Kim Jong Un says he is ready to sit down again...
7,1079638749359886336,28785486,2018-12-31 07:21:51,https://abcn.ws/2EWgnJZ,Note says Britain's queen 'naturally' wanted n...,NaN,A newly discovered note in the U.K.'s National...
8,1083043497228361729,28785486,2019-01-09 16:51:06,https://abcn.ws/2CXslBU,"'Green Book,' 'Bohemian Rhapsody,' 'Roma' take...",NaN,The 2019 Golden Globes were definitely one for...
11,1083952075099303936,28785486,2019-01-12 05:01:28,https://abcn.ws/2D4KZYz,"Everything you need to know about 'Baby Shark,...",NaN,Parents can't get it out of their heads and ki...
...,...,...,...,...,...,...,...
43288,1084579301318709248,6107422,2019-01-13 22:33:50,https://www.thetimes.co.uk/article/caitlin-mor...,Caitlin Moran: the politics of mega TVs,NaN,Sixty-five inch flat-screen HD TV. Sixty-five ...
43293,1083029513188139014,6107422,2019-01-09 15:55:32,https://www.thetimes.co.uk/article/the-wall-th...,The wall that helped Donald Trump win the pres...,NaN,The wall that Donald Trump used to help win th...
43294,1079684112397537281,6107422,2018-12-31 10:22:06,https://www.thetimes.co.uk/article/older-royal...,Older royals carry the weight of public duties,NaN,The Duke and Duchess of Cambridge on Christmas...
43298,1081819771144937472,6107422,2019-01-06 07:48:27,https://www.thetimes.co.uk/article/the-best-ho...,The best holidays in France for 2019: our ulti...,NaN,Welcome to our ultimate guide to the world’s b...


In [37]:
nt_moreonec['news_text'].nunique()

4210

Existem 4117 (antes 6311) 'news_text' diferentes dentro dos que são repetidos.

In [38]:
nt_moreonec['news_text'].value_counts().value_counts()

2     2990
3      725
4      233
5      122
6       61
7       27
8       17
9        6
10       5
11       5
13       4
18       2
12       2
16       2
14       2
21       2
22       1
20       1
17       1
15       1
19       1
Name: news_text, dtype: int64

In [39]:
news_text_VC = nt_moreonec['news_text'].value_counts()
texts = news_text_VC.index # textos que ocorrem mais que uma vez em 'news_text'

for i in range(len(texts)):
    # para cada texto localiza todas as notícias com esse 'news_text'
    # e cria Série com os respectivos índices no dataset items e os 'news_id's correspondentes
    id2replace = items.news_id.loc[items['news_text']==texts[i]]  # index | news_id

    for index, value in id2replace.items():
        # subsititui os id's para 1 único no items
        items.loc[index,'news_id'] = id2replace.iloc[0]
        
        # índices dos comentários para cada 'news_id' a substituir
        idxs = ireplies.loc[ireplies['in_reply_to_news_id'] == value].index
        #substitui os id's para 1 único no ireplies
        for idx in idxs:
            ireplies.loc[idx,'in_reply_to_news_id'] = id2replace.iloc[0]



In [40]:
# Por fim ficaram id's de notícias duplicados. Removemo-os.
items.drop_duplicates(subset=['news_id'], inplace=True)

len(items)

24165

### Missing values

In [41]:
items_missing_v = items.isnull().sum()
print(items_missing_v)

news_id               0
newsoutlet_id         0
news_tweet_date       0
news_url              0
new_title            72
news_date          8540
news_text           233
dtype: int64


In [42]:
items.isnull().value_counts()

news_id  newsoutlet_id  news_tweet_date  news_url  new_title  news_date  news_text
False    False          False            False     False      False      False        15624
                                                              True       False         8308
                                                                         True           160
                                                   True       True       True            72
                                                   False      False      True             1
dtype: int64

Caso *news_title=True and news_date=True and news_text=True* => 72

In [43]:
# Drop das rows com missing values em 'news_text' 
null_text = items[items['news_text'].isnull()]

items.drop(null_text.index, inplace=True)

In [44]:
items_missing_v = items.isnull().sum()
print(items_missing_v)

news_id               0
newsoutlet_id         0
news_tweet_date       0
news_url              0
new_title             0
news_date          8308
news_text             0
dtype: int64


In [45]:
# export do csv

outlets.to_csv('outlets.csv', index=False)
ireplies.to_csv('ireplies.csv', index=False)
items.to_csv('items.csv', index=False)

**Export dos datasets concluido com sucesso.**